In [4]:
import numpy as np
import readScreen
import agent

SIM_FRAMES = 5

ImportError: No module named 'readScreen'

In [1]:
def getState():
    state = np.zeros((5,1))
    
    state[0,0] = readScreen.getQlength(upper)
    state[1,0] = readScreen.getQlength(lower)
    state[2,0] = readScreen.getQlength(left)
    state[3,0] = readScreen.getQlength(right)
    
    state[4,0] = agent.getTLSphase()
    
    return state

In [3]:
def makeMove(state,action):
    if action == 1:
        agent.setTLSphase((agent.getTLSphase + 1)%8)
           
    agent.simulateFrames(SIM_FRAMES)
    newState = getState()
        
    return newState

In [5]:
def getReward(state):
    qLengths = state[:4]
    reward = (-1) * np.sum(qLengths) * np.std(qLengths)
    
    return reward

In [6]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import RMSprop

Using TensorFlow backend.


In [7]:
model = Sequential()
model.add(Dense(164, kernel_initializer='lecun_uniform', input_shape=(5,)))
model.add(Activation('relu'))
#model.add(Dropout(0.2)) I'm not using dropout, but maybe you wanna give it a try?

model.add(Dense(150, kernel_initializer='lecun_uniform'))
model.add(Activation('relu'))
#model.add(Dropout(0.2))

model.add(Dense(2, kernel_initializer='lecun_uniform'))
model.add(Activation('linear')) #linear output so we can have range of real-valued outputs

rms = RMSprop()
model.compile(loss='mse', optimizer=rms)

In [8]:
model.predict(state.reshape(1,5), batch_size=1)
#just to show an example output; read outputs left to right: up/down/left/right

NameError: name 'state' is not defined

In [9]:
model.compile(loss='mse', optimizer=rms)#reset weights of neural network
epochs = 3000
gamma = 0.975
epsilon = 1
batchSize = 40
buffer = 80
replay = []
#stores tuples of (S, A, R, S')
h = 0
for i in range(epochs):
    
    state = getState() #using the harder state initialization function
    status = 1
    #while game still in progress
    while(status < 10000):
        #We are in state S
        #Let's run our Q function on S to get Q values for all possible actions
        qval = model.predict(state.reshape(1,5), batch_size=1)
        if (random.random() < epsilon): #choose random action
            action = np.random.randint(0,2)
        else: #choose best action from Q(s,a) values
            action = (np.argmax(qval))
        #Take action, observe new state S'
        new_state = makeMove(state, action)
        #Observe reward
        reward = getReward(new_state)
        
        #Experience replay storage
        if (len(replay) < buffer): #if buffer not filled, add to it
            replay.append((state, action, reward, new_state))
        else: #if buffer full, overwrite old values
            if (h < (buffer-1)):
                h += 1
            else:
                h = 0
            replay[h] = (state, action, reward, new_state)
            #randomly sample our experience replay memory
            minibatch = random.sample(replay, batchSize)
            X_train = []
            y_train = []
            for memory in minibatch:
                #Get max_Q(S',a)
                old_state, action, reward, new_state = memory
                old_qval = model.predict(old_state.reshape(1,5), batch_size=1)
                newQ = model.predict(new_state.reshape(1,5), batch_size=1)
                maxQ = np.max(newQ)
                y = np.zeros((1,2))
                y[:] = old_qval[:]
                update = (reward + (gamma * maxQ))
                y[0][action] = update
                X_train.append(old_state.reshape(5,))
                y_train.append(y.reshape(2,))
            
            X_train = np.array(X_train)
            y_train = np.array(y_train)
            print("Frame #: %s" % (i,))
            model.fit(X_train, y_train, batch_size=batchSize, nb_epoch=1, verbose=1)
            state = new_state
        status += 1
        clear_output(wait=True)
    if epsilon > 0.1: #decrement epsilon over time
        epsilon -= (1/epochs)

NameError: name 'initGridPlayer' is not defined